In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch

# Check if GPU is available
print("CUDA Available:", torch.cuda.is_available())

# Print current device
print("Current Device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Show available GPU
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    print("CUDA Version:", torch.version.cuda)
    print("PyTorch Version:", torch.__version__)

import torch
torch.cuda.empty_cache()

CUDA Available: False
Current Device: cpu


In [3]:
# storage for hyperparameters used in the model

file_path = "data/kloiya.csv"

# Hyperparameters

# for input 
sequence_length = 28 # a week per sequence
input_size = 1 # flow rate is one feature 

#for model
hidden_size = 64
num_layers = 3
output_size = 1 

# for training
batch_size = 128
learning_rate = 0.0003
num_epochs = 80

# setting up for CUDA
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
import torch
import torch.nn as nn

class LSTM_fr_model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()

        self.hidden_size = hidden_size 
        self.num_layers = num_layers

        # Defined LSTM layer using torch.nn
        self.lstm = nn.LSTM(input_size,hidden_size,num_layers, batch_first=True) #this will create the LSTM with pytorch

        self.fc = nn.Linear(hidden_size,output_size) #it will map the last hidden into an output

    # writing the forward pass
    def forward(self,x):
        batch_size= x.shape[0]

        # initializes hidden and cell states with the correct shape
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)

        # Pass input through LSTM, discard hidden and cell states
        lstm_out, (hn, cn) = self.lstm(x, (h0, c0))

        # Extracting the last timestep's output 
        last_out = lstm_out[:,-1,:]

        # Passing through the fully connected layer
        output = self.fc(last_out)

        return output

In [6]:
import torch 
import pandas as pd
import numpy as np

class FlowRate(): #flow rate function that creates the sequences inside the batch. 
    def __init__(self, data, sequence_length):
        self.sequence_length = sequence_length
        self.data = data

    def __len__(self): # this method returns the number of sequences that can be made with the data with a counting principle
        return len(self.data) - self.sequence_length

    def __getitem__(self, index):
        x = self.data[index : index + self.sequence_length] #get the sequence
        y = self.data[index + self.sequence_length] # gets the target value to calculate loss
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32) # This returns tuple (sequence,target)


def load_data(my_data,sequence_length,batch_size,shuffle = "True"): #due to missing values in the dataset, linear interpolation must be used
    # read the data out of flow rate and linearly interpolate between missing dates
    df = pd.read_csv(my_data, parse_dates=["Date"], date_format="%Y/%m/%d")
    df.set_index("Date",inplace=True)
    
    # Check for duplicate dates
    if df.index.duplicated().any():
        print("Removing duplicates...")
        df = df[~df.index.duplicated(keep="first")]  # Keep first occurrence

    # create the full range of data, with empty slots to interpolate
    new_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq="D")
    df = df.reindex(new_range)
    df["Value"] = df["Value"].interpolate(method="linear")    

    # convert into a numpy array
    num_data = df["Value"].values

    dataset = FlowRate(num_data,sequence_length)
    dataloader = torch.utils.data.DataLoader(dataset,batch_size=batch_size,shuffle=False) # we dont want to shuffle sequential data
    
    return dataloader



In [7]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split

# Load dataset
file_path = "kloiya.csv"
dataset = load_data(file_path, sequence_length, batch_size)

# Extract raw data from DataLoader
raw_data = []
for batch in dataset:
    raw_data.extend(batch[0].cpu().numpy().flatten())  # Convert to NumPy

dataset = np.array(raw_data)

# Prepare sequences for LSTM
X, y = [], []
for i in range(len(dataset) - sequence_length):
    X.append(dataset[i:i + sequence_length])
    y.append(dataset[i + sequence_length])  

X = np.array(X)
y = np.array(y)


print(f"Before split: X shape: {X.shape}, y shape: {y.shape}")
# Split train (70%) and train+valid set (30%) (which will be further split)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False)

# Then, split test+valid into validation (15%) and test (15%)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)

# normalize all data 
train_min, train_max = X_train.min(), X_train.max()
X_train = (X_train - train_min) / (train_max - train_min)
y_train = (y_train - train_min) / (train_max - train_min)

valid_min, valid_max = X_valid.min(), X_valid.max()
X_valid = (X_valid - valid_min) / (valid_max - valid_min)
y_valid = (y_valid - valid_min) / (valid_max - valid_min)

test_min, test_max = X_test.min(), X_test.max()
X_test = (X_test - test_min) / (test_max - test_min)
y_test = (y_test - test_min) / (test_max - test_min)

# Convert to tensors 
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_valid = torch.tensor(X_valid, dtype=torch.float32)
y_valid = torch.tensor(y_valid, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

#  Move to GPU for GPU training 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train, y_train = X_train.to(device), y_train.to(device)
X_valid, y_valid = X_valid.to(device), y_valid.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

# Print Data Shapes
print(f"✅ X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"✅ X_valid shape: {X_valid.shape}, y_valid shape: {y_valid.shape}")
print(f"✅ X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

Before split: X shape: (595084, 28), y shape: (595084,)
✅ X_train shape: torch.Size([416558, 28]), y_train shape: torch.Size([416558])
✅ X_valid shape: torch.Size([89263, 28]), y_valid shape: torch.Size([89263])
✅ X_test shape: torch.Size([89263, 28]), y_test shape: torch.Size([89263])


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import os

# move data to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train, y_train = X_train.to(device), y_train.to(device)
X_valid, y_valid = X_valid.to(device), y_valid.to(device)

#set up model using parameters from config
model = LSTM_fr_model(input_size, hidden_size, num_layers, output_size).to(device)

# loss & optim 
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# scheduler 
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=10, factor=0.3, threshold=5e-4, min_lr=5e-7)

# early stopping
early_stopping_patience = 10  # Stop training if no improvement for 10 epochs
best_valid_loss = float('inf')
early_stopping_counter = 0
best_model_path = "best_model.pth"  # Path to save best model

# training loop
for epoch in range(num_epochs):  # loop over the epochs (outer)
    model.train()  # set model to training mode
    epoch_losses = 0  # track total loss per epoch

    for i in range(0, len(X_train), batch_size):  # inner loop over mini-batches
        x_batch = X_train[i:i + batch_size]
        y_batch = y_train[i:i + batch_size]

        # Ensure correct shape for LSTM
        x_batch = x_batch.view(x_batch.shape[0], sequence_length, input_size)

        # Move batch to GPU
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()  # zero the gradients before backpropagation
        
        # Forward pass
        outputs = model(x_batch)

        # Compute loss
        loss = criterion(outputs, y_batch.view(-1, 1))

        # Backpropagation
        loss.backward()
        optimizer.step()

        # running total of loss for epoch
        epoch_losses += loss.item()

    # Compute average loss for the epoch
    train_loss = epoch_losses / (len(X_train) // batch_size)

    # Validation loss  
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        X_valid = X_valid.view(X_valid.shape[0], sequence_length, input_size)  # Ensure correct shape
        X_valid = X_valid.to(device)
        valid_predictions = model(X_valid).cpu().numpy().flatten()
        valid_actuals = y_valid.cpu().numpy().flatten()
    
    valid_loss = mean_squared_error(valid_actuals, valid_predictions)

    #per-epoch log
    print(f"\Epoch [{epoch+1}/{num_epochs}]")
    print(f"Training Loss: {train_loss:.6f}")
    print(f"Validation Loss: {valid_loss:.6f}")

    # RMSE, MAE, coefficient of determination
    rmse = np.sqrt(valid_loss)
    mae = mean_absolute_error(valid_actuals, valid_predictions)
    r2 = r2_score(valid_actuals, valid_predictions)

    print(f"RMSE: {rmse:.4f} | MAE: {mae:.4f} | R² Score: {r2:.4f}")

    # check for early stop
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        early_stopping_counter = 0  # Reset counter
        torch.save(model.state_dict(), best_model_path)  # Save best model
        print(f"Model improved. Saving new best model to {best_model_path}")
    else:
        early_stopping_counter += 1
        print(f"No improvement. Early stopping counter: {early_stopping_counter}/{early_stopping_patience}")

    if early_stopping_counter >= early_stopping_patience:
        print("\nEarly stop, loading best model.")
        model.load_state_dict(torch.load(best_model_path, weights_only=True))  # FIXED
        break

    # Update learning rate scheduler
    scheduler.step(valid_loss)


📊 Epoch [1/80]
✅ Training Loss: 0.001176
✅ Validation Loss: 0.001749
📉 RMSE: 0.0418 | MAE: 0.0237 | R² Score: 0.6213
💾 Model improved! Saving new best model to best_model.pth

📊 Epoch [2/80]
✅ Training Loss: 0.000596
✅ Validation Loss: 0.001242
📉 RMSE: 0.0352 | MAE: 0.0195 | R² Score: 0.7311
💾 Model improved! Saving new best model to best_model.pth

📊 Epoch [3/80]
✅ Training Loss: 0.000444
✅ Validation Loss: 0.001381
📉 RMSE: 0.0372 | MAE: 0.0225 | R² Score: 0.7009
🛑 No improvement. Early stopping counter: 1/10

📊 Epoch [4/80]
✅ Training Loss: 0.000366
✅ Validation Loss: 0.000696
📉 RMSE: 0.0264 | MAE: 0.0175 | R² Score: 0.8493
💾 Model improved! Saving new best model to best_model.pth

📊 Epoch [5/80]
✅ Training Loss: 0.000285
✅ Validation Loss: 0.000586
📉 RMSE: 0.0242 | MAE: 0.0156 | R² Score: 0.8731
💾 Model improved! Saving new best model to best_model.pth

📊 Epoch [6/80]
✅ Training Loss: 0.000263
✅ Validation Loss: 0.000613
📉 RMSE: 0.0247 | MAE: 0.0162 | R² Score: 0.8674
🛑 No improvem

In [8]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# Load the best model 
best_model_path = "best_model.pth"  # Ensure this path matches the saved model in training
model = LSTM_fr_model(input_size, hidden_size, num_layers, output_size).to(device)
model.load_state_dict(torch.load(best_model_path,map_location=torch.device('cpu'), weights_only=False)) # move to CPu
model.to(device)
model.eval()  # Set to evaluation mode

# Run predictions on the test set
with torch.no_grad():
    X_test = X_test.view(X_test.shape[0], sequence_length, input_size)  # Ensure correct shape
    X_test = X_test.to(device)  # Move to GPU if available
    predictions = model(X_test).cpu().numpy().flatten() * (test_max - test_min) + test_min # Convert to 1D NumPy array
    actuals = y_test.cpu().numpy().flatten() * (test_max - test_min) + test_min  # Convert to 1D NumPy array

# Run predictions on the validation set
with torch.no_grad():
    X_valid = X_valid.view(X_valid.shape[0], sequence_length, input_size)  # Ensure correct shape
    X_valid = X_valid.to(device)  # Move to GPU if available
    valid_predictions = model(X_valid).cpu().numpy().flatten() * (valid_max - valid_min) + valid_min # Convert to 1D NumPy array
    valid_actuals = y_valid.cpu().numpy().flatten() * (valid_max - valid_min) + valid_min  # Convert to 1D NumPy array

#denormalize the training set
y_train_denormalized = y_train.cpu().numpy().flatten() * (train_max - train_min) + train_min

# Handle potential division by zero in MAPE calculation
nonzero_mask = actuals != 0
if np.any(nonzero_mask):
    mape = np.mean(np.abs((actuals[nonzero_mask] - predictions[nonzero_mask]) / actuals[nonzero_mask])) * 100
else:
    mape = np.nan  # If all actual values are zero, MAPE is not defined

# Compute Metrics
mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actuals, predictions)
r2 = r2_score(actuals, predictions)

#Print Evaluation Metrics
print("\nModel Evaluation Metrics (Best Model Loaded):")
print(f"MSE (Mean Squared Error): {mse:.4f}")
print(f"RMSE (Root Mean Squared Error): {rmse:.4f}")
print(f"MAE (Mean Absolute Error): {mae:.4f}")
print(f"MAPE (Mean Absolute Percentage Error): {mape:.2f}%")
print(f"R² Score: {r2:.4f}")

#Create time indices
train_size = len(y_train)
valid_size = len(y_valid)
test_size = len(y_test)

train_index = np.arange(0, train_size)
valid_index = np.arange(train_size, train_size + valid_size)
test_index = np.arange(train_size + valid_size, train_size + valid_size + test_size)

%pylab qt
plt.figure(figsize=(12, 6))
# Define the start and end dates for the x-axis
years = np.linspace(1964, 2022, num=6, dtype=int)  # Generate 6 year ticks
x_positions = np.linspace(0, 595083, num=6)  # Match them with x-axis positions

# Create the plot
fig, ax = plt.subplots()

ax.plot(train_index, y_train_denormalized, label="Training Data", color="blue", alpha=1, linewidth=1.2)
ax.plot(valid_index, valid_actuals, label="Actual Validation Data", color="purple", linestyle="solid", alpha=0.9, linewidth=1.2)
ax.plot(test_index, actuals, label="Actual Test Data", color="green", linestyle="solid", alpha=0.9, linewidth=1.2)
ax.plot(test_index, predictions, label="Predicted Test Data", color="red", linestyle="dashed", alpha=0.7, linewidth=1.2)

# Set x-axis labels
ax.set_xticks(x_positions)
ax.set_xticklabels(years)

# Labels and title
ax.set_xlabel("Time (years)")
ax.set_ylabel("Flow Rate")
ax.set_title("Time Series of Test Data with Model Predictions")

# Legend and grid
ax.legend()
ax.grid(True)

# Show the plot
plt.show()


Model Evaluation Metrics (Best Model Loaded):
MSE (Mean Squared Error): 2.4598
RMSE (Root Mean Squared Error): 1.5684
MAE (Mean Absolute Error): 0.6112
MAPE (Mean Absolute Percentage Error): 6.23%
R² Score: 0.9439
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


2025-03-07 17:18:35.173 python[99132:4224548] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-07 17:18:35.173 python[99132:4224548] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [9]:
from IPython.display import FileLink

# Define the path to the saved model
model_path = "best_model.pth"

# Create a download link
FileLink(model_path)

/kaggle/working/best_model.pth